In [1]:
import logging
import sys
import os

logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [2]:
ACCESS_TOKEN_PATH = os.path.pardir + "/api_keys/openai.key"
SAVE_PATH = os.getcwd() + "/document_store/"

In [3]:
from src.communicators import GPTCommunicator

communicator = GPTCommunicator(ACCESS_TOKEN_PATH, "gpt-3.5-turbo")

In [4]:
communicator.post_prompt("Hi")

2024-03-28 00:37:32,485 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Hello! How can I assist you today?'

In [6]:
from src.data_processors import WikiTextProcessor

processor = WikiTextProcessor(
    dataset_version = "wikitext-2-raw-v1", 
    split = "train", 
    communicator = communicator,
    verbose = True
)
passages = processor.process_text(
    token_limit = 2048, 
    save_path = SAVE_PATH,
    save_filename = "data.csv",
    manipulate_pattern = ("Star Trek", "Star Wars"),
)
passages[0]

2024-03-28 00:37:53,923 - INFO - 629 passages created. 
2024-03-28 00:37:55,038 - INFO - 211 passages remaining after limiting tokens
2024-03-28 00:37:55,163 - INFO - largest passage after trim is 2025 tokens
2024-03-28 00:37:55,164 - INFO - 3 passages manipulated; 'Star Trek' -> 'Star Wars'
2024-03-28 00:37:55,176 - INFO - Processed data saved to: /Users/dev/projects/Rag_DocumentQA/document_store/data.csv


" = Gambia women 's national football team = \n\n\n The Gambia women 's national football team represents the Gambia in international football competition . The team , however , has not competed in a match recognised by FIFA , the sport 's international governing body , despite that organised women 's football has been played in the country since 1998 . The Gambia has two youth teams , an under @-@ 17 side that has competed in FIFA U @-@ 17 Women 's World Cup qualifiers , and an under @-@ 19 side that withdrew from regional qualifiers for an under @-@ 19 World Cup . The development of a national team faces challenges similar to those across Africa , although the national football association has four staff members focusing on women 's football . \n\n\n = = The team = = \n\n\n In 1985 , few countries had women 's national football teams . While the sport gained popularity worldwide in later decades , the Gambia 's national team only played its first game in 2007 . That game was not FIFA

In [7]:
from src.vectorstore_handlers import LangchainVectorstore
from langchain_community.embeddings import HuggingFaceEmbeddings

vs = LangchainVectorstore(
    embedding_type = HuggingFaceEmbeddings(),
    processed_csv_path = SAVE_PATH+"data.csv",
    verbose_info = True
)

vs.create_local_vectorstore(save_path=SAVE_PATH)

vs.create_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5
    }
)
vs.retrieve_top_documents(query="What show is the episode 'Marauders' from?")

2024-03-28 00:38:17,667 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2024-03-28 00:38:17,966 - INFO - Use pytorch device: cpu
2024-03-28 00:38:17,974 - INFO - Vectorstore and retriever must be set using the class methods.
2024-03-28 00:38:21,192 - INFO - Creating a new local vectorstore at: /Users/dev/projects/Rag_DocumentQA/document_store/


Processing documents:   0%|          | 0/211 [00:00<?, ?it/s]

2024-03-28 00:38:21,726 - INFO - Loading faiss.
2024-03-28 00:38:21,738 - INFO - Successfully loaded faiss.


Processing documents: 100%|██████████| 211/211 [00:35<00:00,  5.90it/s]

2024-03-28 00:38:56,967 - INFO - Vectorstore successfully set and saved to /Users/dev/projects/Rag_DocumentQA/document_store/
2024-03-28 00:38:56,967 - INFO - Retriever successfully set


[': 76\ntext: = Marauders ( Star Wars : Enterprise ) = \n\n\n " Marauders " is the sixth episode of the second season of the American science fiction television series Star Wars : Enterprise , the 32nd episode overall . It first aired on October 30 , 2002 , on the UPN network within the United States . The story was created by executive producers Rick Berman and Brannon Braga with a teleplay by David Wilcox . A similar premise had been included in the original pitch for Star Wars by Gene Roddenberry . \n\n Set in the 22nd century , the series follows the adventures of the first Starfleet starship Enterprise , registration NX @-@ 01 . In this episode , while in search of deuterium , Enterprise discovers a mining colony that is being controlled by Klingons who are bullying the inhabitants and hoarding their supplies . The crew conduct repairs on the colony and train the colonists to fight off the Klingons . \n\n This episode was mostly filmed on location in a quarry in Ventura County , C